In [1]:
import cv2
import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [6]:

image = cv2.imread("./thermal_images/train/fully_filled/thermal_image_1.png")


# gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# normalized_image = cv2.normalize(gray_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)


# thermal_image = cv2.applyColorMap(normalized_image, cv2.COLORMAP_HOT)  

cv2.imshow("Thermal Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
def rgb_to_thermal(rgb_image):

    rgb_array = np.array(rgb_image)
    rgb_normalized = rgb_array / 255.0

    thermal_array = np.zeros_like(rgb_normalized)
    thermal_array[:, :, 0] = 0.5 * rgb_normalized[:, :, 0]  # Red channel
    thermal_array[:, :, 1] = 0.0  # Green channel
    thermal_array[:, :, 2] = 0.5 * (1 - rgb_normalized[:, :, 0])  # Blue channel

    # Convert the NumPy array back to a PIL Image
    thermal_image = Image.fromarray((thermal_array * 255).astype(np.uint8))

    return thermal_image

In [8]:
import os
from torch.utils.data import Dataset
from PIL import Image

class ThermographyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = os.listdir(os.path.join(root_dir, 'train'))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)} 
        self.images = []
        for cls_name in self.classes:
            cls_dir = os.path.join(root_dir, 'train', cls_name)
            for img_name in os.listdir(cls_dir):
                img_path = os.path.join(cls_dir, img_name)
                self.images.append((img_path, self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image) 

        return image, label


In [14]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(128),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ThermographyDataset(root_dir='./thermal_images/', transform=transform)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)


In [26]:
decodeLabel=['fully_filled','half_filled']

In [15]:
import torch
import torch.nn as nn

class ThermoImageClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(ThermoImageClassifier, self).__init__()
        self.conv1 = nn.Sequential(

            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )
        self.fc1 = nn.Linear(32 * 64 * 64, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [16]:
model = ThermoImageClassifier(num_classes=2)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [18]:
for images, labels in dataloader:
    print(labels)

tensor([1, 1, 0, 1])
tensor([1, 1, 1, 1])
tensor([0, 0, 1, 0])
tensor([1, 1, 0, 1])
tensor([1, 1, 0, 1])
tensor([1, 0, 0, 1])
tensor([1, 1, 1, 0])
tensor([1, 0, 0, 1])
tensor([0, 1, 1, 0])
tensor([1, 1, 1, 1])
tensor([1, 1, 1, 1])
tensor([0, 1, 1, 1])
tensor([0, 1, 1, 0])
tensor([0, 0, 0, 1])
tensor([1, 0, 1, 1])
tensor([1, 1, 0, 1])
tensor([1, 1, 1, 0])
tensor([0, 1, 1, 0])
tensor([0, 1, 1, 1])
tensor([1, 1, 0, 1])
tensor([0, 1, 1, 1])
tensor([1, 1, 0, 1])
tensor([0, 0, 0, 0])
tensor([1, 1, 0, 1])
tensor([0, 1, 0, 0])
tensor([0, 1, 1, 0])
tensor([0, 1, 1, 0])
tensor([1, 1, 1, 1])
tensor([1, 0, 0, 0])
tensor([0, 1, 1, 1])
tensor([1, 0, 1, 1])
tensor([0, 1, 0, 1])
tensor([0, 1, 1, 1])
tensor([0, 1, 1, 1])
tensor([0, 1, 0, 1])
tensor([0, 0, 0, 1])
tensor([1, 1, 0, 0])
tensor([0, 1, 1, 1])
tensor([1, 1, 1, 1])
tensor([1, 0, 0, 1])
tensor([1, 1, 1, 0])
tensor([1, 0, 1, 1])
tensor([0, 1, 1, 0])
tensor([0, 0, 0, 1])
tensor([0, 0, 1, 1])
tensor([1, 1, 1, 0])
tensor([1, 1, 1, 1])
tensor([1, 1,

KeyboardInterrupt: 

In [27]:
epochs = 1
for epoch in range(epochs):
    running_loss = 0.0
    for step, (images, labels) in enumerate(dataloader, 1):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print loss for each step
        print(f"Epoch {epoch + 1}/{epochs}, Step {step}/{len(dataloader)}, Loss: {loss.item()}")

    # Print average loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs} - Average Loss: {running_loss / len(dataloader)}")

# Save trained model (optional)
# torch.save(model.state_dict(), "model.pt")


Epoch 1/1, Step 1/6063, Loss: 0.0
Epoch 1/1, Step 2/6063, Loss: 0.0
Epoch 1/1, Step 3/6063, Loss: 0.0
Epoch 1/1, Step 4/6063, Loss: 0.0
Epoch 1/1, Step 5/6063, Loss: 0.0
Epoch 1/1, Step 6/6063, Loss: 0.0
Epoch 1/1, Step 7/6063, Loss: 0.0
Epoch 1/1, Step 8/6063, Loss: 0.0
Epoch 1/1, Step 9/6063, Loss: 0.0
Epoch 1/1, Step 10/6063, Loss: 0.0
Epoch 1/1, Step 11/6063, Loss: 0.0
Epoch 1/1, Step 12/6063, Loss: 0.0
Epoch 1/1, Step 13/6063, Loss: 0.0
Epoch 1/1, Step 14/6063, Loss: 0.0
Epoch 1/1, Step 15/6063, Loss: 0.0
Epoch 1/1, Step 16/6063, Loss: 0.0
Epoch 1/1, Step 17/6063, Loss: 0.0
Epoch 1/1, Step 18/6063, Loss: 0.0
Epoch 1/1, Step 19/6063, Loss: 0.0
Epoch 1/1, Step 20/6063, Loss: 0.0
Epoch 1/1, Step 21/6063, Loss: 0.0
Epoch 1/1, Step 22/6063, Loss: 0.0
Epoch 1/1, Step 23/6063, Loss: 0.0
Epoch 1/1, Step 24/6063, Loss: 0.0
Epoch 1/1, Step 25/6063, Loss: 0.0
Epoch 1/1, Step 26/6063, Loss: 0.0
Epoch 1/1, Step 27/6063, Loss: 0.0
Epoch 1/1, Step 28/6063, Loss: 0.0
Epoch 1/1, Step 29/6063, Loss

In [28]:
model.eval()

ThermoImageClassifier(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=131072, out_features=2, bias=True)
)

In [29]:
# sample_image_path = './data/train/full_filled/pipeimage.jpg'
sample_image_path = './thermal_images/test/half_filled/thermal_image_10.png'
sample_image = Image.open(sample_image_path).convert('RGB')
thermal=rgb_to_thermal(sample_image)
sample_image = transform(thermal)
sample_image.shape


torch.Size([3, 128, 128])

In [30]:
with torch.no_grad():
    output = model(sample_image.unsqueeze(0))

predicted_class = torch.argmax(output).item()
print(f'Predicted class: {decodeLabel[predicted_class]}')

Predicted class: half_filled
